In [1]:
#pip install langchain langchain-openai langchain-groq langchain-community requests pandas yfinance

In [2]:
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env file

load_dotenv("apis.env")
# Set up API keys

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(
    model_name="gpt-4",  # or "gpt-4" "gpt-3.5-turbo" for a less expensive option
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY")
    )

from langchain_groq import ChatGroq

# For LLaMA 3 8B model

groq_llm_llama3_8b = ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0,
    api_key=os.getenv("GROQ_API_KEY")
    )

# For LLaMA 3 70B model

groq_llm_llama3_70b = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0,
    api_key=os.getenv("GROQ_API_KEY")
    )

In [4]:
#Create Selection Function:

#def get_llm(provider="openai", model="default"):
def get_llm(provider, model="default"):
    if provider.lower() == "openai":
        return openai_llm
    elif provider.lower() == "groq":
        if model.lower() == "llama3-70b":
            return groq_llm_llama3_70b
        else:
            return groq_llm_llama3_8b
    else:
        return openai_llm  # default

In [5]:
import os
import requests
import langchain
import yfinance as yf
from langchain.tools import Tool
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
import json
import numpy as np

In [6]:
def get_stock_price(symbol):
    """
    Fetches the latest stock price for the given symbol.
    
    Args:
        symbol (str): Stock ticker symbol (e.g., 'AAPL')
        
    Returns:
        dict: Stock information including current price and daily change
    """
    try:
        stock = yf.Ticker(symbol)
        
        # Get current price from latest historical data
        hist = stock.history(period="1d")
        if hist.empty:
            return {"error": f"No data available for {symbol}"}
        current_price = hist["Close"].iloc[-1]
        
        # Get previous close price from metadata
        previous_close = stock.info.get("regularMarketPreviousClose")
        
        # Fallback: Use 2-day history if metadata unavailable
        if not previous_close:
            hist_2d = stock.history(period="2d")
            previous_close = hist_2d["Close"].iloc[-2] if len(hist_2d) > 1 else current_price
        
        # Calculate daily changes
        daily_change = current_price - previous_close
        daily_change_pct = (daily_change / previous_close) * 100
        
        return {
            "symbol": symbol,
            "price": round(current_price, 2),
            "change": round(daily_change, 2),
            "change_pct": round(daily_change_pct, 2)
        }
        
    except Exception as e:
        return {"error": f"Error retrieving stock price for {symbol}: {str(e)}"}


In [7]:
def rebalance_portfolio(portfolio_str):
    """
    Takes a portfolio string representation and suggests rebalancing actions.

    Args:
        portfolio_str (str): String representation of portfolio, e.g., "{'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}"

    Returns:
        str: Rebalancing recommendations
    """
    try:
        # Parse the portfolio string into a dictionary
        portfolio = json.loads(portfolio_str.replace("'", '"'))  # Convert single quotes to double quotes for JSON parsing
        
        # Validate that weights sum to 1
        total_weight = sum(portfolio.values())
        if not (0.99 <= total_weight <= 1.01):  # Allow for minor floating-point inaccuracies
            return f"Error: Portfolio weights must sum to 1. Current total weight: {total_weight:.2f}"

        # Calculate target weight (equal weight for this assignment)
        num_assets = len(portfolio)
        target_weight = 1 / num_assets

        # Suggest buying or selling actions to achieve balance
        recommendations = []
        for symbol, weight in portfolio.items():
            if weight < target_weight:
                recommendations.append(f"Buy {symbol}: Increase weight by {round(target_weight - weight, 4)}")
            elif weight > target_weight:
                recommendations.append(f"Sell {symbol}: Decrease weight by {round(weight - target_weight, 4)}")
            else:
                recommendations.append(f"{symbol} is balanced.")

        # Return recommendations in a clear format
        return "\n".join(recommendations)

    except Exception as e:
        return f"Error analyzing portfolio: {str(e)}"

In [8]:
from pydantic import BaseModel
from langchain.tools import StructuredTool
import yfinance as yf
import numpy as np

# Market Trend Analysis Function
def market_trend_analysis():
    try:
        ticker_symbol = "SPY"
        stock_data = yf.Ticker(ticker_symbol)
        hist = stock_data.history(period="5d")

        if hist.empty:
            return f"No data available for {ticker_symbol}."

        closing_prices = hist["Close"]
        start_price = closing_prices.iloc[0]
        end_price = closing_prices.iloc[-1]
        five_day_return = ((end_price - start_price) / start_price) * 100

        daily_returns = closing_prices.pct_change().dropna()
        volatility = np.std(daily_returns) * np.sqrt(252)

        trend_summary = (
            f"Market Trend Analysis for {ticker_symbol} (S&P 500):\n"
            f"- 5-Day Return: {five_day_return:.2f}%\n"
            f"- Annualized Volatility: {volatility:.2f}%\n"
            f"- Closing Prices (Last Week):\n{closing_prices.to_string(index=True)}"
        )
        return trend_summary

    except Exception as e:
        return f"Error analyzing market trends: {str(e)}"

# Input Schema (even if unused)
class MarketTrendInput(BaseModel):
    dummy: str = "none"  # Required for compatibility with LangChain

# Create StructuredTool
trend_tool = Tool(
    name="MarketTrendAnalysis",
    description="Analyzes market trends over the past week.",
    func=lambda _: market_trend_analysis()#,
    # args_schema=MarketTrendInput
)

In [9]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

# Define input schema for each tool
#class StockPriceInput(BaseModel):
    #symbol: str

#class RebalancePortfolioInput(BaseModel):
    #portfolio_str: str

# Define tools with args_schema
stock_price_tool = Tool(
    name="StockPriceLookup",
    description="Fetches stock price data.",
    func=get_stock_price#,
    #args_schema=StockPriceInput  # Specify input schema
)

rebalance_tool = Tool(
    name="PortfolioRebalancer",
    description="Suggests portfolio rebalancing actions.",
    func=rebalance_portfolio#,
    #args_schema=RebalancePortfolioInput  # Specify input schema
)

In [10]:
import time

def create_and_run_agent(llm_provider, llm_model, query):
    """Runs the agent and returns both response and execution time."""
    llm = get_llm(llm_provider, llm_model)
    tools = [stock_price_tool, rebalance_tool, trend_tool]
    
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=False
    )
    
    start_time = time.perf_counter()
    response = agent.run(query)
    end_time = time.perf_counter()
    
    return response, end_time - start_time

In [11]:
def prompt(portfolio):
    prompt = """
    You are a financial assistant with access to the below tools. Your task is to - 
    - Retrieve stock prices for assets in a given portfolio if available 
    - Check if the portfolio is imbalanced based on an equal-weight strategy
    - Recommend actions if the portfolio needs rebalancing

    1. MarketTrendAnalyzer:
    - Description: Use this to analyze the recent S&P 500 market trend. This function takes NO input. Call it with: MarketTrendAnalyzer

    2. PortfolioRebalancer:
    - Description: Use this to check if a portfolio is balanced and suggest rebalancing actions. This function takes a single argument: a stringified dictionary where keys are stock tickers and values are portfolio weights (must sum to ~1.0).
    - Example input: '{'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}'
    - Call it like this: PortfolioRebalancer('{'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}')
    - The actual portfolio might have more than 3 stocks. It's not an error. The key in the dict-like input is the stock. The number following ':" is the % of the portfolio that stock makes up. Understand the pattern in the input. Don't throw an error if there are more/less than 3 stocks.

    3. StockPriceLookup:
    - Description: Use this to fetch the latest price of any stock. This function takes a single string argument representing the ticker symbol.
    - Example input: "GOOG"
    - Call it like this: StockPriceLookup("GOOG"). 
    - Important - When looking up on yfinance, drop the quotes from the string. And look for GOOG. Don't throw can't find 'GOOG' error erroneously.
    - If you are not able to look up stock price, don't let it keep you from doing the main task. Use other tools and other stock information you have to accomplish the task.
    
    Analyze this portfolio and recommend changes"
""" + f"{portfolio}"

    
    return prompt


In [12]:
# Test cases with different LLMs

def run_test_cases():

    user_portfolio_1 = {"AAPL": 0.50, "TSLA": 0.30, "GOOGL": 0.20} 

    user_portfolio_2 = {"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}

    user_portfolio_3 = {"NVDA": 0.90, "TSLA": 0.10}  # Highly imbalanced
    
    user_portfolio_4 = {"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}  # Near-equal weights
    
    user_portfolio_5 = {"BTC": 1.0}  # Single asset

   

    # Test with OpenAI

    print("OpenAI GPT-4 Results:")

    print("Portfolio 1:", create_and_run_agent("openai", "default",  prompt(user_portfolio_1)))

    print("Portfolio 2:", create_and_run_agent("openai", "default",  prompt(user_portfolio_2)))

    print("Portfolio 3:", create_and_run_agent("openai", "default",  prompt(user_portfolio_3)))

    print("Portfolio 4:", create_and_run_agent("openai", "default",  prompt(user_portfolio_4)))

    print("Portfolio 5:", create_and_run_agent("openai", "default",  prompt(user_portfolio_5)))

   

    # Test with Groq LLaMA3-8B

    print("\nGroq LLaMA3-8B Results:")

    print("Portfolio 1:", create_and_run_agent("groq", "llama3-8b", prompt(user_portfolio_1)))

    print("Portfolio 2:", create_and_run_agent("groq", "llama3-8b", prompt(user_portfolio_2)))

    print("Portfolio 3:", create_and_run_agent("groq", "llama3-8b", prompt(user_portfolio_3)))

    print("Portfolio 4:", create_and_run_agent("groq", "llama3-8b", prompt(user_portfolio_4)))

    print("Portfolio 5:", create_and_run_agent("groq", "llama3-8b", prompt(user_portfolio_5)))

   

    # Test with Groq LLaMA3-70B

    print("\nGroq LLaMA3-70B Results:")

    print("Portfolio 1:", create_and_run_agent("groq", "llama3-70b", prompt(user_portfolio_1)))

    print("Portfolio 2:", create_and_run_agent("groq", "llama3-70b", prompt(user_portfolio_2)))

    print("Portfolio 3:", create_and_run_agent("groq", "llama3-70b", prompt(user_portfolio_3)))

    print("Portfolio 4:", create_and_run_agent("groq", "llama3-70b", prompt(user_portfolio_4)))

    print("Portfolio 5:", create_and_run_agent("groq", "llama3-70b", prompt(user_portfolio_5)))




if __name__ == "__main__":

    run_test_cases()

OpenAI GPT-4 Results:


C:\Users\HP\AppData\Local\Temp\ipykernel_38836\3701203217.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\HP\AppData\Local\Temp\ipykernel_38836\3701203217.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)
404 Client Error: Not Found fo

Portfolio 1: ("The portfolio needs rebalancing. Sell 'AAPL' to decrease its weight by 0.1667. Buy 'TSLA' to increase its weight by 0.0333. Buy 'GOOGL' to increase its weight by 0.1333.", 22.487648000009358)


$'MSFT': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'NVDA': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'AMZN': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'META': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Portfolio 2: ('The portfolio is balanced. No rebalancing actions are needed.', 26.96740389999468)
Portfolio 3: ('The portfolio is imbalanced. The recommended actions are to sell NVDA and decrease its weight by 0.4 and to buy TSLA and increase its weight by 0.4.', 13.970653699943796)


$'MSFT': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'GOOGL': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'AMZN': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Portfolio 4: ("The portfolio needs rebalancing. Increase the weights of 'MSFT' and 'GOOGL' by 0.0033 each, and decrease the weight of 'AMZN' by 0.0067.", 16.327134700026363)
Portfolio 5: ('The portfolio is balanced and does not need any changes.', 7.088647800032049)

Groq LLaMA3-8B Results:
Portfolio 1: ('The portfolio needs rebalancing, and the suggested actions are to sell AAPL, buy TSLA, and buy GOOGL. However, before taking any action, we should consider the feasibility of these actions based on the current stock prices.', 5.790897800005041)
Portfolio 2: ('Agent stopped due to iteration limit or time limit.', 160.67697899998166)
Portfolio 3: ('The portfolio is imbalanced, with NVDA making up 90% of the portfolio and TSLA making up 10%. To rebalance the portfolio, I would recommend selling some NVDA and buying some TSLA to bring the weights closer to an equal-weight strategy.', 21.405110400053672)


$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly deliste

Portfolio 4: ('Agent stopped due to iteration limit or time limit.', 119.16669549990911)
Portfolio 5: ('The portfolio should be rebalanced by adding traditional stocks, such as Apple (AAPL), to diversify the risk.', 14.001763600041158)

Groq LLaMA3-70B Results:
Portfolio 1: ('To rebalance the portfolio, sell AAPL to decrease its weight by 0.1667, buy TSLA to increase its weight by 0.0333, and buy GOOGL to increase its weight by 0.1333.', 6.040600199950859)
Portfolio 2: ('The portfolio is currently balanced and does not require any rebalancing actions.', 23.98715209995862)
Portfolio 3: ('Based on the analysis, the portfolio is imbalanced and needs rebalancing. To achieve an equal-weight strategy, I recommend selling NVDA and decreasing its weight by 0.4, and buying TSLA and increasing its weight by 0.4. This will result in a more balanced portfolio with equal weights of 0.5 for both NVDA and TSLA.', 25.709412499913014)
Portfolio 4: ('Based on the analysis, the portfolio is imbalanced an

In [26]:
# Test cases with different LLMs

def run_test_cases():
    
    user_portfolio_4 = {"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}  # Near-equal weights
    

    # Test with Groq LLaMA3-8B

    print("\nGroq LLaMA3-8B Results:")

    print("Portfolio 4:", create_and_run_agent("groq", "llama3-8b", prompt(user_portfolio_4)))


if __name__ == "__main__":

    run_test_cases()


Groq LLaMA3-8B Results:


$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$'{"MSFT": 0.33, "GOOGL": 0.33, "AMZN": 0.34}': possibly deliste

Portfolio 4: ('Agent stopped due to iteration limit or time limit.', 69.95127560000401)
